# Prediction of Transcription Factories: A Machine Learning Blueprint
### Shaurya Jauhari, Mora Lab, GZHMU.

Transcription Factories are those purported sites that have been identified as the breeding grounds for transcripts. Their relevance heightens when the spatial organization of the genome is considered. Since transcription factories attract distant segments of the genome together for inter-mingling and possibly carrying out regulation, true picture emerges of the parts that co-localize and co-express. This has been a major lacuna in the dogma of the current suite of enrichment tools for genomic regions.

## Prerequisites

The protocol would require the availability of the following tools. Kindly ensure that they have been successfully installed in your system and embrace a **PATH** variable definition.
1. [samtools](http://www.htslib.org/)
2. [deeptools](https://deeptools.readthedocs.io/en/develop/)

Additionally, the notebook employs several bash scripts that are available [here](https://github.com/shauryajauhari/transcriptionFactoriesGREG/tree/master/terminalScripts), and R functions available [here](https://github.com/shauryajauhari/transcriptionFactoriesGREG/tree/master/projectFunctions). These scripts hold the relevant annotation and must be cloned to the project directory for implementation.

## Setting up Data

We have compiled a tabulation for the metadata associated with cell types in GREG. The relevant features shall be the basis for our machine learning model, but before even reaching that far we're tempted to visualize the reads from these features (transcription factors, histone marks, and RNA-Seqs).

In [1]:
## Install package to read excel file and load library.
if (!require(readxl)) install.packages("readxl", dependencies = TRUE)
library(readxl)
  
## Import the master table.
masterData <- read_excel("siftedData.xlsx")

## Identifying individual download links for the features
masterData$`Download Link` <- strsplit(masterData$`Download Link`, ",")

## For consistency and cleanliness, let's remove whitespaces from the column.
  for (i in 1: nrow(masterData))
  {
    for (j in 1: length(masterData$`Download Link`[[i]]))
    {
     lapply(masterData$`Download Link`[[i]][j], trimws)
    }
  }

head(masterData)

Loading required package: readxl



Cell Type,Feature,Samples,Source,Species,Genome Version,Laboratory,Notes,Download Link,File Type,Selection Criteria
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<list>,<chr>,<chr>
A549,CTCF,GSM1003582,ENCODE,Homo sapiens,hg19,"Bradley Bernstein, Broad","GSM1003582 is from Bradley Bernstein, Broad and the sample is treated with 100 nM dexamethasone for 1 hour","https://www.encodeproject.org/files/ENCFF631JUR/@@download/ENCFF631JUR.bam , https://www.encodeproject.org/files/ENCFF685FVQ/@@download/ENCFF685FVQ.bam",BAM,More samples from this lab; more chosen samples from this method (treatment with 100 nM dexamethasone for 1 hour).
A549,EP300,GSM1010827,ENCODE,Homo sapiens,hg19,"Myers Lab, Hudson Alpha Institute of Biotechnology.",GSM1010827 is treated with 0.02% ethanol for 1 hour,"https://www.encodeproject.org/files/ENCFF540LLB/@@download/ENCFF540LLB.bam , https://www.encodeproject.org/files/ENCFF864VSA/@@download/ENCFF864VSA.bam",BAM,Only choice
A549,H3K27me3,GSM1003577,ENCODE,Homo sapiens,hg19,"Bradley Bernstein, Broad","GSM1003577 is from Bradley Bernstein, Broad and the sample is treated with 100 nM dexamethasone for 1 hour","https://www.encodeproject.org/files/ENCFF800NPS/@@download/ENCFF800NPS.bam , https://www.encodeproject.org/files/ENCFF664RTA/@@download/ENCFF664RTA.bam",BAM,More chosen samples from this method (treatment with 100 nM dexamethasone for 1 hour)
A549,H3K36me3,GSM1003494,ENCODE,Homo sapiens,hg19,"Bradley Bernstein, Broad","GSM1003494 is from Bradley Bernstein, Broad and the sample is treated with 100 nM dexamethasone for 1 hour","https://www.encodeproject.org/files/ENCFF386GAK/@@download/ENCFF386GAK.bam , https://www.encodeproject.org/files/ENCFF572FCV/@@download/ENCFF572FCV.bam",BAM,More chosen samples from this method (treatment with 100 nM dexamethasone for 1 hour)
A549,H3K4me1,GSM1003495,ENCODE,Homo sapiens,hg19,"Bradley Bernstein, Broad","GSM1003495 is from Bradley Bernstein, Broad and the sample is treated with 100 nM dexamethasone for 1 hour","https://www.encodeproject.org/files/ENCFF892UTA/@@download/ENCFF892UTA.bam , https://www.encodeproject.org/files/ENCFF000AIE/@@download/ENCFF000AIE.bam, https://www.encodeproject.org/files/ENCFF739OOI/@@download/ENCFF739OOI.bam, https://www.encodeproject.org/files/ENCFF000AIG/@@download/ENCFF000AIG.bam",BAM,More chosen samples from this method (treatment with 100 nM dexamethasone for 1 hour)
A549,H3K4me2,GSM1003511,ENCODE,Homo sapiens,hg19,"Bradley Bernstein, Broad","GSM1003511 is from Bradley Bernstein, Broad and the sample is treated with 100 nM dexamethasone for 1 hour","https://www.encodeproject.org/files/ENCFF874VXV/@@download/ENCFF874VXV.bam , https://www.encodeproject.org/files/ENCFF927EWB/@@download/ENCFF927EWB.bam",BAM,More chosen samples from this method (treatment with 100 nM dexamethasone for 1 hour)


In [2]:
## Identifying cell-types and features
cells <- unique(masterData$`Cell Type`)
cells

[1] "A549"    "H1_hESC" "HeLa_S3" "IMR90"   "K562"    "MCF7"

In [3]:
features <- unique(masterData$Feature)
features

[1] "CTCF"     "EP300"    "H3K27me3" "H3K36me3" "H3K4me1"  "H3K4me2" 
 [7] "H3K4me3"  "H3K9ac"   "H3K9me3"  "RAD21"    "RNAPol2"  "RNAPol3" 
[13] "RNA-Seq"  "YY1"

### Initializing Data Store

In [9]:
## We shall create a directory named 'GREG' that will house all the data related to all the features associated with
## each cell type, as defined above.

# current project directory as working directory.
setwd(".")

# Parent directory
dir.create("GREG")

# Create subfolders by the name of cell-types. 
for (i in 1:length(cells))
  {
    dir.create(paste0("./GREG/",cells[i]))
  }


# Create sub-subfolders by the name of features.
for (i in 1:length(cells))
  {
    for(j in 1:length(features))
      {
        dir.create(paste0(paste0("./GREG/",cells[i]),paste0("/",features[j])))
      }
  }

The *tree* command helps visualize a directory structure. It is available by default in linux systems, while in MacOS it can be explicitly installed with *brew*; the syntax is **$ brew install tree**. 

In [10]:
system("tree")

![](directoryStructureGREG.png)

Now that we have the directory structure in place, we shall commence with the downloading of the relevant alignment files, to their appropriate location. The files can be referenced from the column *Download Link* in the table *masterData*. Note that there are **NA** instances in the table, that'll be dealt in accordance. 

In [ ]:
source("./projectFunctions/downloadBAMfiles.R")

## This function takes two input values, one is the cell-type and the other is the feature. If the user wishes to 
## download BAM files for a particular pair of cell-type and feature, the following code (for example) will do that
## and store the files in the appropriate folder location.

# downloadBAMfiles("A549", "RAD21")

## Alternatively, if the user would like to download all files relative to all cell-types and all features 
## (as enlisted in the table above), we can structure the code like this.

for (cell in cells)
  {
    for (feature in features)
    {
        downloadBAMfiles(cell, feature)
    }
}

## Setting up Target Genomic Regions

In cognizance to [GREG](http://www.dx.doi.org/10.1093/database/baz162), we define genomic regions with a 2 kilobase bin size. These shall be the pivots corresponding to which the features with read counts shall be hinged. Additionally, we want to make our BED file [1-based](http://genome.ucsc.edu/FAQ/FAQformat.html#format1), to align with the format in GREG. 

In [2]:
## Source the UCSC library for Homo sapiens genome, version hg19.
BiocManager::install("BSgenome.Hsapiens.UCSC.hg19")
library(BSgenome.Hsapiens.UCSC.hg19)

Bioconductor version 3.11 (BiocManager 1.30.10), R 4.0.2 (2020-06-22)

Installing package(s) 'BSgenome.Hsapiens.UCSC.hg19'

installing the source package ‘BSgenome.Hsapiens.UCSC.hg19’


Old packages: 'LOLA', 'RcppArmadillo', 'TTR', 'bit64', 'callr', 'conquer',
  'glue', 'inline', 'jsonlite', 'mgcv', 'mnormt', 'processx', 'tidyr', 'vctrs',
  'zip'

Loading required package: BSgenome

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.c

From the chromosome size definitions available in the loaded package, we create an index as follows. This shall be the premises on which our genomic bins shall be carved out.

In [4]:
si <- seqinfo(BSgenome.Hsapiens.UCSC.hg19)
si

Seqinfo object with 298 sequences (2 circular) from hg19 genome:
  seqnames           seqlengths isCircular genome
  chr1                249250621      FALSE   hg19
  chr2                243199373      FALSE   hg19
  chr3                198022430      FALSE   hg19
  chr4                191154276      FALSE   hg19
  chr5                180915260      FALSE   hg19
  ...                       ...        ...    ...
  chr21_gl383580_alt      74652      FALSE   hg19
  chr21_gl383581_alt     116690      FALSE   hg19
  chr22_gl383582_alt     162811      FALSE   hg19
  chr22_gl383583_alt      96924      FALSE   hg19
  chr22_kb663609_alt      74013      FALSE   hg19

The function **tileGenome** allows to structure a definitive GenomicRanges object.

In [5]:
Bins2k <-  tileGenome(si, tilewidth = 2000, cut.last.tile.in.chrom = TRUE)
Bins2k

GRanges object with 1617579 ranges and 0 metadata columns:
                      seqnames      ranges strand
                         <Rle>   <IRanges>  <Rle>
        [1]               chr1      1-2000      *
        [2]               chr1   2001-4000      *
        [3]               chr1   4001-6000      *
        [4]               chr1   6001-8000      *
        [5]               chr1  8001-10000      *
        ...                ...         ...    ...
  [1617575] chr22_kb663609_alt 66001-68000      *
  [1617576] chr22_kb663609_alt 68001-70000      *
  [1617577] chr22_kb663609_alt 70001-72000      *
  [1617578] chr22_kb663609_alt 72001-74000      *
  [1617579] chr22_kb663609_alt 74001-74013      *
  -------
  seqinfo: 298 sequences (2 circular) from hg19 genome

In [4]:
targetBED <- as.data.frame(Bins2k)
targetBED <- targetBED[, 1:3]
colnames(targetBED) <- c("chrom", "start", "end")
head(targetBED)
write.table(targetBED, file = "hg19_2k_bins.bed", sep = "\t", quote = FALSE, col.names = FALSE, row.names = FALSE)

,chrom,start,end
,<fct>,<int>,<int>
1,chr1,1,2000
2,chr1,2001,4000
3,chr1,4001,6000
4,chr1,6001,8000
5,chr1,8001,10000
6,chr1,10001,12000


These are the regions where the read counts need to be mapped, and will be in use with the *computeMatrix* execution down the line.

## Feature Files 

For exemplifying, let's pick up ChIP-Seq data for epigenetic marks - H3K4me1, H3K4me2, H3K4me3, and H3K27ac for the H1 hESC cell line.(WIP; More instances to follow)

![](siftingBAM.jpg)

Since our desired function for mapping coverages to the genomic bins, mandates execution on atleast two files, the catch here is that there could be samples where only single replicate exists. Also, since we plan to average read counts from multiple replicates eventually, we shall create a copy of the alignment file if it exists in singleton. 

In [4]:
## The following script examines the feature files (BAM files in subfolders under each cell-type). 
## If they exist in replicates (two or more), then we can proceed with multiBAMSummary as is, else it'll create a copy
## of the single BAM file.

## system("bash ./terminalScripts/adjustBAMFiles.sh <path>)

for (cell in cells)
  {
    for (feature in features)
    {
      ## Run adjustBAMFiles.sh
       system(paste("bash ./terminalScripts/adjustBAMFiles.sh",
       paste(paste0(paste0("./GREG/",cell), paste0("/", feature)))))
    }
  }

The BAM files need to be sorted so that they follow a continuous order of locationing in the genome. The script *samtoolsSortRun.sh* will help us do that.

In [4]:
## export PATH=$PATH:<path/to/samtools>

## system("bash ./terminalScripts/samtoolsSortRun.sh <path>")

for (cell in cells)
  {
    for (feature in features)
    {
      ## Run samtoolsSortRun.sh
        system(paste("bash ./terminalScripts/samtoolsSortRun.sh",
        paste(paste0(paste0("./GREG/",cell), paste0("/", feature, "/*.bam")))))
    }
  }

Next, these BAM files need to be indexed. The script *samtoolsIndexRun.sh* helps just do that. The path to the BAM files is to be specified as an argument to this script. There could be a single or multiple BAM files available.
If there are multiple BAM files in a common folder, include the argument as "*.bam". Alternatively, if you have BAM files in different folders, they can be fed to the script as under, seperated with a single space. Note that the index files are created in the same location as the BAM file. [They] just need to exist; we hardly require to process them.

In [5]:
## export PATH=$PATH:<path/to/samtools>

## system("bash ./terminalScripts/samtoolsIndexRun.sh <path>")

for (cell in cells)
  {
    for (feature in features)
    {
      ## Run samtoolsIndexRun.sh
       system(paste("bash ./terminalScripts/samtoolsIndexRun.sh",
       paste(paste0(paste0("./GREG/",cell), paste0("/", feature, "/*.bam")))))
    }
  }

Now that we have the index files along, we proceed towards getting read counts for our defined bin-size. We achieve this via deeptools suite, a function called *multiBAMSummary*.

In [ ]:
## The bash script can be viewed at the terminal.
## Another instance of the same script - "deepToolsBAMSummaryGREG.sh" can be used to take as input all BAM files
## representing replicates in a feature (same folder) and output the read counts over an interval of 2Kb.

for (cell in cells)
  {
    for (feature in features)
    {
      ## Run deepToolsmultiBAMSummaryGREG.sh
        system("bash ./terminalScripts/deepToolsmultiBAMSummaryGREG.sh", 
       paste0("./GREG/",cell, "/", feature), 
               intern = FALSE, 
               ignore.stdout = FALSE, 
               ignore.stderr = FALSE, 
               wait = TRUE)
    }
  }

Having the read counts for all replicates, for all features, and for all cell-types, we shall now proceed with assembling output of *multiBamSummary*(*readCounts.tab*) from each feature, merging it together for each cell-type, and generate a table for normalized (BPM) read counts. The function **postProcessBamFiles** aids to do just that.

In [ ]:
source("./projectFunctions/postProcessBamFiles.R") ## loading function definition

install.packages("plyr") # installing package for use with the function.

for (cell in cells)
    {
      postProcessBamFiles(cell) # for all cell-types
    }

We examine the proportion at which reads are observed in a given bin. Basically we want to observe reads per million per nucleotide (RPM), or, as deepTools define it, [Bins Per Million (BPM)](https://deeptools.readthedocs.io/en/develop/content/tools/bamCoverage.html) mapped reads, same as TPM in RNA-seq. We calculate the number of reads that fall into a bin divided by the number of reads in the feature data set, then multiply by 10^6 to get per million. In this way you get an RPM track of 2 kilobase bins covering the genome, thus samples with different numbers of reads become comparable. 

The above function saves the bin-intervals and the count matrix for all features, for each cell-type in its home folder. It must be noted that not all cell-types shall have homogeneity in the number of features (since source files are available for selected features currently), and the number of bins (a valid quantum of score might not be available for selected bins as emanated from *multiBamSummary*). Let us recall the ones for **A549** and **K562** for example.  

In [15]:
a549 <- read.table("./GREG/A549/normalizedReads.txt", header = TRUE)
head(a549)

,CTCF,EP300,H3K27me3,H3K36me3,H3K4me1,H3K4me2,H3K4me3,H3K9ac,H3K9me3,RAD21,RNAPol2,YY1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,0.00000000,0.00000000,0.00000,0,0.00000000,0.000000,0.00000000,0.0000000,0.00000000,0.00000000
2,0,0,0.00000000,0.00000000,0.00000,0,0.00000000,0.000000,0.00000000,0.0000000,0.00000000,0.00000000
3,0,0,0.00000000,0.00000000,0.00000,0,0.00000000,0.000000,0.00000000,0.0000000,0.00000000,0.00000000
4,0,0,0.00000000,0.00000000,0.00000,0,0.00000000,0.000000,0.00000000,0.0000000,0.00000000,0.00000000
5,0,0,0.00000000,0.00000000,0.00000,0,0.00000000,0.000000,0.00000000,0.0000000,0.00000000,0.00000000
6,0,0,0.02727825,0.02613314,1.58534,0,0.03331304,1.537907,0.02891425,0.1631014,0.06050439,0.05796767


In [19]:
nrow(a549)

[1] 1547861

In [18]:
binsRegions <- read.table("./GREG/A549/binsRegions.txt", header = TRUE)
head(binsRegions)

,X..chr.,X.start.,X.end.
,<fct>,<int>,<int>
1,chr1,1,2000
2,chr1,2001,4000
3,chr1,4001,6000
4,chr1,6001,8000
5,chr1,8001,10000
6,chr1,10001,12000


In [26]:
nrow(binsRegions)

[1] 1547861

In [24]:
k562 <- read.table("./GREG/K562/normalizedReads.txt", header = TRUE)
head(k562)

,CTCF,EP300,H3K27me3,H3K36me3,H3K4me1,H3K4me2,H3K4me3,H3K9ac,H3K9me3,RAD21,RNA.Seq,RNAPol2,RNAPol3,YY1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.00000,0.0000000,0.0000000,0.000000,0.0000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,0.0000000,0.00000,0.0000000
2,0.00000,0.0000000,0.0000000,0.000000,0.0000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,0.0000000,0.00000,0.0000000
3,0.00000,0.0000000,0.0000000,0.000000,0.0000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,0.0000000,0.00000,0.0000000
4,0.00000,0.0000000,0.0000000,0.000000,0.0000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,0.0000000,0.00000,0.0000000
5,0.00000,0.0000000,0.0000000,0.000000,0.0000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,0.0000000,0.00000,0.0000000
6,47.61676,0.0274939,0.8087214,1.045595,0.7726153,1.202883,1.627348,1.188197,6.28546,8.497575,0,0.1668389,20.66886,0.1562263


In [25]:
nrow(k562)

[1] 1518172

Now, we're good to proceed. We shall tap *computeMatrix* function from **deepTools** for preparing input for *plotProfiles* function from the same suite.

## computeMatrix and plotProfile functions: visualization of peaks

Since computeMatrix algorithm takes bigWig files as input, we shall first convert our bedGraph files to bigWig files. For doing so, we shall download and install *bedGraphToBigWig* and *bedClip* tools from **http://genome.ucsc.edu**, and chromosome sizes for hg19. Remember to make the tools executable. Alternatively, if these tools are already present inyour system, they must be defined in the **PATH** variable.

In [12]:
## Change to the working directory
## Make tools executable

system ("chmod a+x ./bedGraphToBigWig")
system ("chmod a+x ./bedClip")


## Trim the extremities for genomic coordinates to be contained inside the defined chromosome sizes 
## for the given version of human genome (hg19 here).

system("find . -name "*.bedGraph" -exec bedClip {} hg19.chrom.sizes {}.clipped \;")

## Sort our bedGraph files

system ("find . -name "*.clipped" -exec sort -k1,1 -k2,2n {} -o {}.sorted \;")

## Converting to bigWig format

system("find . -name "*.sorted" -exec bedGraphToBigWig {} hg19.chrom.sizes {}.bw \;")

The chromosomal lengths (***hg19.chrom.sizes*** above) for the hg19 version of the human genome can be fetched from this [link](https://genome.ucsc.edu/goldenPath/help/hg19.chrom.sizes).

On compleletion of the pipeline hitherto, each cell-type shall have their reported features harboring,
1. Replicate files (BAM) and their indexes.
2. Each iteration of files upon normalization (bedGraph files, sourced from *readCounts.tab*), outputs from *bedClip* and *sort* tools.
3. Finally, the bigWig file.

The following is an instance from the **MCF7** cell line for **CTCF**, **EP300**, and **H3K27me3** features.

![](mcf7.png)

Next, again in line with the GREG format, we are going to tranform the hg19 coordinates to hg38 system. To do that, we shall use **[CrossMap](http://crossmap.sourceforge.net/)** tool. The versatility of this tool is refreshing and we can employ it for a variety of files. Here, we convert the bigWig files for each feature.

In [4]:
system("pip3 install Crossmap") ## Installing tool

system("find . -name "*.bw" -exec CrossMap.py bigwig hg19ToHg38.over.chain.gz {} {}.lift2hg38 \;")

For the purpose of plotting, the bigWig files cover the whole genome (all 2Kb bins for all features in the cell-types), but eventually the regions we’ll be interested to see the coverage distribution will be the ones derived from distance based clustering algorithms, applied to GREG data. Despite the fact that we’re measuring the counts for the whole genome (which by the way might come in handy for our ML task), consequently, we shall have the plots for the bin-hubs and LR-hubs only. Actually, we can visualize both these regions as overlays.

P.S. LR-Hubs are long range interactions tracked between two communities.

In [ ]:
## Create an instance
intervals <- targetBED

## Extract length of each chromosome type.
levs <- levels(intervals$chrom)

for (item in levs)
{
  occur =0
  for(len in 1:nrow(intervals))
    {
      if (intervals$chrom[len]== item)
        {
          occur = occur + 1
          intervals$binIds[len] <- paste0("Bin", as.character(occur))
        }
    }
  cat("The chromosome", item, "has", occur, "occurences. \n")
}

## Save file
intervals$binsGREGformat <- paste0(intervals$chr,":",intervals$binIds)
write.table(intervals, file = "intervalsMasterReferenceGREG.txt", sep = "\t", row.names = FALSE, quote = FALSE)

In [3]:
head(intervals)

,chr,start,end,binIds,binsGREGformat
,<fct>,<int>,<int>,<fct>,<fct>
1,chr1,1,2000,Bin1,chr1:Bin1
2,chr1,2001,4000,Bin2,chr1:Bin2
3,chr1,4001,6000,Bin3,chr1:Bin3
4,chr1,6001,8000,Bin4,chr1:Bin4
5,chr1,8001,10000,Bin5,chr1:Bin5
6,chr1,10001,12000,Bin6,chr1:Bin6


The **binsGREGformat** is the "key" column for referencing data; literally.

The reason we create this master reckoner is that we shall be matching the hubs (with bins in GREG format- chromosome:binID) to this table and derive the chromosome, it's start and end indices, for all regions, that will go as input for *- R* (regions file) input to *computeMatrix* tool.

### Determining LR-hubs and Bin-hubs

#### LR-hubs

The clustering results are loaded into the session, and the coordinates of the hubs shall be determined out of these.

In [ ]:
## Filtering Hubs for different cell types.
## The cell-types in questions are all existing celltypes in GREG, except for 
## IPS6.9 and IPS19.11, due to lack of feature data associated with them.

## Loading clustering results from GREG.
load("./data/GREG_cluster_v3.RData")

We know that IPS cells have been excluded due to lack of feature data, and so only the residual types shall be brought up for analysis.

The variable **Long_Range_relationships** carries cell-type information and the LR IDs, amongst others that we shall gloss over for now. We will extract these bits and frame a new variable.

In [ ]:
## Define valid cell types
cellTypes <- c("IMR90", "MCF7", "HELA", "K562", "H1ESC", "A549")
validLongRangeRelationships <- Long_Range_relationships[Long_Range_relationships$r_CellType %in% cellTypes, ]

## From this variable, we shall only be interested in the LR IDs( for which we shall fetching details from another
## variable), and celltypes. Let's prune the data in that accord.

validLongRangeRelationships <- validLongRangeRelationships[,c(1,6)]

In [ ]:
## Next step is to filter LR IDs for each given celltype as a exclusive dataset.
LRdataCellTypes <- lapply(cellTypes, function(ct) {validLongRangeRelationships[validLongRangeRelationships$r_CellType %in% ct, ]})
names(LRdataCellTypes) <- cellTypes


## Finding bin ids for given LR IDs.
LRids <- names(Bins_genes_per_LR_relationship)

Now, we shall extract bins associated with these LR IDs. A variable **Bins_genes_per_LR_relationship** incubates bin IDs for all corresponding long range interactions.

In [ ]:
dataLRids <- list()
for (count in 1:length(LRids))
{
  dataLRids[[count]] <- eval(parse(text = paste0("Bins_genes_per_LR_relationship", 
                                               "$", LRids[count], "[,1]")))
}

names(dataLRids) <- LRids


## Let us create third and fourth columns to store bin ids and indices, 
## respectively.


for (i in 1:length(LRdataCellTypes))
  {
    LRdataCellTypes[[i]][c(3, 4)] <- NA
    colnames(LRdataCellTypes[[i]]) <- c("LR_id", "r_CellType", "binIds", "binIndices")
  }

Exploring IDs for all cells.

In [ ]:
## So, we now have the list of lists of bin ids that are associated with the given
## LR ids. We shall now map these to the cell types.


for (i in 1:length(LRdataCellTypes))
  {
    for (j in 1:nrow(LRdataCellTypes[[i]])) 
      {
          if(LRdataCellTypes[[i]][[1]][[j]] %in% names(dataLRids))
            {
              
              LRdataCellTypes[[i]][[3]][[j]] <- dataLRids[LRdataCellTypes[[i]][[1]][[j]]]
            }
      }
  }

Our goal for this module has been to derive bin intervals for bin IDs.

In [ ]:
## For "binIndices" now.
## Let us now pull the information of bin indices from the master chart.


for (i in 1:length(LRdataCellTypes))
{
  for (j in 1:nrow(LRdataCellTypes[[i]])) 
  {
    for(k in 1:length(LRdataCellTypes[[i]][[3]][[j]][[1]]))
    {
      if(LRdataCellTypes[[i]][[3]][[j]][[1]][k] %in% intervals$binsGREGformat) 
      {
        LRdataCellTypes[[i]][[4]][[j]][[k]] <- intervals[intervals$binsGREGformat 
                                                          == LRdataCellTypes[[i]][[3]][[j]][[1]][k], c(1, 2, 3)]   
      }
    }
  }
}


Here, we revise the list of lists format to list of dataframes. The "chr", "start", and "end" have to be concatenated together as a row and not each as a sublist; this makes the data structure more fluid.

In [ ]:
## Reformatting the bin Indices - list of dataframes to a consolidated dataframe for all LR IDs.

for (i in 1:length(LRdataCellTypes))
{
  for (j in 1:nrow(LRdataCellTypes[[i]])) 
  {
    LRdataCellTypes[[i]][[4]][[j]] <- do.call(rbind, LRdataCellTypes[[i]][[4]][[j]])
  }
}

Let's save our the variable and optimise its data too.

In [ ]:
## Save this LR-hub tabulation
save(LRdataCellTypes, file = "LRhubsGREG.Rdata")


## We can remove the cell-names and bin-Ids now for convenience.

for (i in 1:length(LRdataCellTypes))
  {
    LRdataCellTypes[[i]]["r_CellType"]<- c() # names of cell-lines
    LRdataCellTypes[[i]]["binIds"]<- c() # bin_Ids
  }


We will design our storage tree for cell-specific hubs.

In [ ]:
## Now that we have the bin indices for all LR-hubs and cell-types, we will be extracting these regions for both 
## those categories.

## Structuring directories.

dir.create("./results") ## create a results folder

## subfolder for cell specific information 

for (i in 1:length(cellTypes))
{
  dir.create(paste0("./results/",cellTypes[i]))
}

cats <- c("LR-hubs", "Bin-hubs")

## sub-subfolders for all LR-hubs

for (i in 1:length(cellTypes))
{
  for(j in 1:length(cats))
  {
    dir.create(paste0("./results/",cellTypes[i], "/", cats[j]))  
  }
}


In [ ]:
## Saving files to appropriate locations

for(i in 1:length(LRdataCellTypes))
{
  for(j in 1:nrow(LRdataCellTypes[[i]]))
  {
      write.table(LRdataCellTypes[[i]][[2]][[j]], 
                  file = paste0("./results/", names(LRdataCellTypes)[[i]], 
                                "/LR-hubs/", LRdataCellTypes[[i]][[1]][[j]], ".bed"), 
                  row.names = FALSE, 
                  col.names = FALSE,
                  quote = FALSE,
                  sep = "\t")
  }
}

##### Total interactions with TFs, lncRNAs, and chr_ranges

In [ ]:
## Create a column to hold sum of number of TFs, lncRNAs, and chromsomal ranges associated with the given bin.

for(i in 1:length(subnetwork_per_LR))
  {
    for(j in 1:length(subnetwork_per_LR[[i]]))
      {
        subnetwork_per_LR[[i]][[j]]["SumTFlncRNAchrRanges"] <- sum(length(subnetwork_per_LR[[i]][[j]][[1]]), # TFs 
                                              length(subnetwork_per_LR[[i]][[j]][[2]]), # lncRNAs
                                              length(subnetwork_per_LR[[i]][[j]][[4]]) ) # chromosome ranges
      }
  }

In [ ]:
## Let us just keep this count and remove all other metrics for convenience.

for(i in 1:length(subnetwork_per_LR))
  {
    for(j in 1:length(subnetwork_per_LR[[i]]))
      {
        subnetwork_per_LR[[i]][[j]] <-   subnetwork_per_LR[[i]][[j]][-c(1:7)]
      }
  }

In [ ]:
## Seperate interaction-count data for each LR id.

listThemAll <- vector(mode="list", length(subnetwork_per_LR))


for(i in 1:length(subnetwork_per_LR))
  {
    for (j in 1:length(subnetwork_per_LR[[i]]))
      {
        listThemAll[[i]][[j]] <- c(names(subnetwork_per_LR[i]),
                   names(subnetwork_per_LR[[i]][j]),
                   subnetwork_per_LR[[i]][[j]][[1]])
      }
  }


## Convert to dataframes.
listThemAllData <- list()

for(i in 1: length(listThemAll))
{
  listThemAllData[[i]] <- as.data.frame(do.call(rbind, listThemAll[[i]]))
  colnames(listThemAllData[[i]]) <- c("LR_id", "binIds", "SumTFlncRNAchrRanges")
}

## Naming indices for easy list access.
names(listThemAllData) <-  names(subnetwork_per_LR)

## Save data.
save(listThemAllData, file = "interactionScore.Rdata")

In [3]:
sessionInfo()

R version 4.0.2 (2020-06-22)
Platform: x86_64-apple-darwin17.0 (64-bit)
Running under: macOS Catalina 10.15.6

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.0/Resources/lib/libRblas.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/4.0/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats4    parallel  stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] BSgenome.Hsapiens.UCSC.hg19_1.4.3 BSgenome_1.56.0                  
 [3] rtracklayer_1.48.0                Biostrings_2.56.0                
 [5] XVector_0.28.0                    GenomicRanges_1.40.0             
 [7] GenomeInfoDb_1.24.2               IRanges_2.22.2                   
 [9] S4Vectors_0.26.1                  BiocGenerics_0.34.0              
[11] readxl_1.3.1                     

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.5                  cellranger_1.1.0           
 [3] pillar